* Look at tag types (from Iterative Parsing Quiz)

In [98]:
import xml.etree.ElementTree as ET
import csv
import codecs
import pprint
import re
from collections import defaultdict

OSM_FILE = "Columbus.osm"

def count_tags(filename):
    tags = {}
    for event, element in ET.iterparse(filename):
        if element.tag not in tags.keys():
            tags[element.tag] = 1
        else:
            tags[element.tag] += 1
    
    return tags

print count_tags(OSM_FILE)

{'node': 675741, 'member': 18913, 'nd': 797507, 'tag': 422176, 'bounds': 1, 'note': 1, 'meta': 1, 'relation': 1124, 'way': 75957, 'osm': 1}


* Look at problem characters (from Tag Types Quiz)

In [99]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys): #count for each type of error keys given regex above
    if element.tag == "tag":
        k = element.attrib['k']
        if re.search(lower, k):
            keys['lower'] += 1
        elif re.search(lower_colon, k):
            keys['lower_colon'] += 1
        elif re.search(problemchars, k):
            keys['problemchars'] += 1
        else:
            keys['other'] +=1
        pass
        
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


keys = process_map(OSM_FILE)
pprint.pprint(keys)

{'lower': 199265, 'lower_colon': 212021, 'other': 10889, 'problemchars': 1}


* Exploring Users from Exploring Users Quiz

In [ ]:
def get_user(element):
    return


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.get('uid'):
            id = element.attrib['uid']
            users.add(id)

    return users

* Create dictionary of keys and values to see which specifically to audit

In [121]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def waynode_kv(osmfile): #add keys to dictionary with each instance of a value
    osm_file = open(osmfile, "r")
    unique_kv = defaultdict(set)
    for i, element in enumerate(get_element(osm_file)):
        if element.tag == "node" or element.tag == "way":                         
            for tag in element.iter("tag"):        
                unique_kv[tag.attrib['k']].add(tag.attrib['v'])    
    print len(unique_kv.keys())  
    return unique_kv

waynode_kv(OSM_FILE)

481


defaultdict(set,
            {'Area': {'brownfield'},
             'Business': {'CTL Engineering'},
             'Category': {'Adult Day Care', 'Recycling Center'},
             'Catering': {'Yes'},
             'Comments': {'Added for OSU G607 SP2012'},
             'FIXME': {'Exactly where is the south end of current trackage?',
              'Is this all Creekside?',
              'check lanes',
              'continue?',
              'detatch from boundary',
              'is the correct maxspeed 25 mph (40 km/h) or 40 mph?',
              'name?',
              'not dual carriageway',
              'rough alignment',
              'validate speed limit change location',
              'validate speed limit is still 65 mph',
              'verify'},
             'FIXME:cycleway': {'verify'},
             'FIXME:name': {'no', 'yes'},
             'FIXME:ref': {'verify'},
             'Fax': {'614-478-5824'},
             'Government': {'State of Ohio - Department of Natural Resource

* Collect last word of addresses to audit street type (from Improving Street Names Quiz)

In [102]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

street_expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in street_expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

audit(OSM_FILE)

defaultdict(set,
            {'101': {'E Dublin Granville Rd #101'},
             '105': {'Mill St Suite 105'},
             '250': {'Chambers Rd #250'},
             '2D': {'Copeland Mill Rd #2D'},
             '320': {'W Old Wilson Bridge Rd #320'},
             '8877': {'8877'},
             '8897': {'8897'},
             'Ave': {'N Cassady Ave'},
             'Blvd': {'Channingway Blvd'},
             'Broadway': {'Broadway'},
             'Center': {'Easton Town Center',
              'Mc Naughten Center',
              'Tremont Center'},
             'Circle': {'Ellington Circle',
              'Gateway Circle',
              'Parkcenter Circle'},
             'Crossing': {'Morse Crossing', 'Red Hook Crossing'},
             'Dr': {'Auto Mall Dr',
              'Humphries Dr',
              'Rentra Dr',
              'Wake Dr',
              'Westpoint Plaza Dr',
              'Westpointe Plaza Dr'},
             'Dr.': {'Centerpoint Dr.'},
             'East': {'Byers Circle Eas

* Create dictionary and replace faulty street names

In [106]:
streetmapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road",
            "Rd": "Road",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "S": "South",
            "SE": "Southeast",
            "SW": "Southwest",
            "N": "North",
            "NW": "Northwest",
            "NE": "Northeast",
            "Pkwy": "Parkway",
            "Rd 101": "Road",
            "St Suite 105": "Street",
            "Rd 250": "Road",
            "Rd 2D": "Road",
            "Rd #320": "Road",
            "8877": "Bad Address",
            "8897": " Bad Address"
            }

def update_name_street(name, streetmapping):
    if name == "E Dublin Granville Rd #101":
        name = "E Dublin Granville Road"
    elif name == "Mill St Suite 105":
        name = "Mill Street"
    elif name == "Chambers Rd #250":
        name = "Chambers Road"
    elif name == "Copeland Mill Rd #2D":
        name = "Copeland Mill Road"
    elif name == "W Old Wilson Bridge Rd #320":
        name = "W Old Wilson Bridge Road"
    
    else: 
        m = street_type_re.search(name) #find if street type in above dictionary
        street_type = m.group()
        if street_type in streetmapping.keys():
            name = re.sub(m.group(), streetmapping[m.group()],name)
    return name

for event, element in ET.iterparse(OSM_FILE, events=("start",)):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "addr:street":
                tag.attrib['v'] = update_name_street(tag.attrib['v'], streetmapping)


* Create dictionary of all zip code values

In [104]:
def is_zip_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zips(osmfile):

    osm_file = open(osmfile, "r")
    zip_codes = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zip_code(tag):                    
                        if tag.attrib['v'] not in zip_codes:
                            zip_codes[tag.attrib['v']] = 1
                        else:
                            zip_codes[tag.attrib['v']] += 1
    return zip_codes

audit_zips('Columbus.osm')

{'35005': 1,
 '43001': 1,
 '43002': 1,
 '43004': 13,
 '43015': 1,
 '43016': 27,
 '43017': 311,
 '43026': 86,
 '43031': 1,
 '43035': 5,
 '43054': 12,
 '43062': 43,
 '43064': 5,
 '43065': 3,
 '43068': 25,
 '43081': 8,
 '43082': 7,
 '43085': 18,
 '43109': 1,
 '43110': 2,
 '43119': 26,
 '43119-8650-08': 1,
 '43123': 18,
 '43125': 5,
 '4313': 1,
 '43137': 1,
 '43140': 2,
 '43147': 56,
 '43162': 4,
 '43201': 66,
 '43202': 48,
 '43203': 3,
 '43204': 11,
 '43205': 10,
 '43206': 17,
 '43207': 16,
 '43209': 24,
 '43210': 88,
 '43211': 4,
 '43212': 76,
 '43213': 11,
 '43214': 46,
 '43215': 78,
 '43215-1430': 1,
 '43216': 1,
 '43219': 50,
 '43220': 9,
 '43220-4800': 1,
 '43221': 110,
 '43222': 3,
 '43223': 2,
 '43224': 16,
 '43227': 7,
 '43228': 30,
 '43229': 15,
 '43230': 13,
 '43231': 17,
 '43232': 5,
 '43235': 77,
 '43240': 16,
 '43328': 1,
 '43802': 1,
 '8765': 1,
 'OH': 1,
 'OH 43206': 1,
 'Ohio': 1}

* Replace faulty zip codes 

In [107]:
def update_zips(name):
    if len(name) < 5:
        print name, 'updated to:'
        name = "Bad Zip"
        print name
        return name
    elif len(name) > 5: #take only first 5 numbers of zip
        print name, 'updated to:'
        if re.search('[0-9]{5}', name):
            updated_name = re.findall('[0-9]{5}', name)
            name = updated_name[0]
            print name
            return name
    
    else:
        return name
       
for event, element in ET.iterparse(OSM_FILE, events=("start",)):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "addr:postcode":
                tag.attrib['v'] = update_zips(tag.attrib['v'])

43220-4800 updated to:
43220
8765 updated to:
Bad Zip
4313 updated to:
Bad Zip
43215-1430 updated to:
43215
OH 43206 updated to:
43206
43119-8650-08 updated to:
43119
Ohio updated to:
Bad Zip
OH updated to:
Bad Zip


* Create dictionary of all city values

In [108]:
def is_city(elem):
    return (elem.attrib['k'] == "addr:city")

def audit_city(osmfile):

    osm_file = open(osmfile, "r")
    cities = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_city(tag):
                    if tag.attrib['v'] not in cities:
                        cities[tag.attrib['v']] = 1
                    else:
                        cities[tag.attrib['v']] += 1
    return cities

audit_city(OSM_FILE) 

{'Adamsville': 1,
 'Alexandria': 1,
 'Amlin': 1,
 'Bexley': 12,
 'Blacklick': 11,
 'Brice': 1,
 'Canal Winchester': 3,
 'Columbus': 782,
 'Columbus, OH': 4,
 'Dublin': 306,
 'Dublin, OH': 15,
 'Etna Township': 1,
 'Gahanna': 7,
 'Galloway': 28,
 'Grandview': 9,
 'Grandview Heights': 6,
 'Grove City': 18,
 'Groveport': 5,
 'Hilliard': 80,
 'Jackson': 1,
 'Lewis Center': 17,
 'London': 2,
 'Marble Cliff': 1,
 'New Albany': 21,
 'Obetz': 4,
 'Pataskala': 22,
 'Pickerington': 56,
 'Pickertington': 1,
 'Plain City': 6,
 'Powell': 2,
 'Reynoldsburg': 26,
 'Reynoldsburh': 1,
 'Upper Arlington': 98,
 'West Jefferson': 3,
 'Westerville': 11,
 'Whitehall': 1,
 'Worthington': 21,
 'columbus': 7,
 'dublin': 1,
 'hilliard': 2}

* Replace faulty city names

In [110]:
citymapping = { "Columbus, OH": "Columbus",
                "Dublin, OH": "Dublin",
                "Reynoldsburh": "Reynoldsburg",
                "Pickertington": "Pickerington"
            }
                
def update_name_city(name, citymapping):
    if name in citymapping:
        print name, "updated to:"
        name = citymapping[name] 
        if name.islower():
            name = name.capitalize()
        print name
        return name

    else:
        if name.islower():
            name = name.capitalize()
        return name

for event, element in ET.iterparse(OSM_FILE, events=("start",)):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "addr:city":
                tag.attrib['v'] = update_name_city(tag.attrib['v'], citymapping)

Columbus, OH updated to:
Columbus
Columbus, OH updated to:
Columbus
Pickertington updated to:
Pickerington
Reynoldsburh updated to:
Reynoldsburg
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Columbus, OH updated to:
Columbus
Columbus, OH updated to:
Columbus
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin
Dublin, OH updated to:
Dublin


* Create dictionary of denominations

In [111]:
def is_church(elem):
    return (elem.attrib['k'] == "denomination")

def audit_church(osmfile):

    osm_file = open(osmfile, "r")
    churches = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_church(tag):
                        if tag.attrib['v'] not in churches:
                            churches[tag.attrib['v']] = 1
                        else:
                            churches[tag.attrib['v']] += 1
                        
    return churches

audit_church(OSM_FILE)

{'Baptist': 1,
 'baptist': 129,
 'brethren': 1,
 'catholic': 22,
 'evangelical': 1,
 'jehovahs_witness': 2,
 'lutheran': 41,
 'messianic_jewish': 2,
 'methodist': 54,
 'mormon': 2,
 'nazarene': 1,
 'orthodox': 1,
 'pentecostal': 10,
 'presbyterian': 18,
 'protestant': 2,
 'roman_catholic': 9,
 'salvation_army': 1,
 'united_church_of_christ': 1,
 'united_methodist': 2,
 'united_methodist;methodist': 1}

* Replace faulty denomination names

In [112]:
denommapping = { "united_methodist;methodist": "methodist",
                 "united_methodist": "methodist"
            }

def first_lower(s):
    return s[0].lower() + s[1:]                 

def update_name_denom(name, denommapping):
    if name.istitle():
        print name, 'updated to:'
        name = name.lower()
        print name
        return name
    elif name in denommapping:
        print name, "updated to:"
        name = denommapping[name]
        print name
        return name

    else:
        return name

for event, element in ET.iterparse(OSM_FILE, events=("start",)):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "denomination":
                tag.attrib['v'] = update_name_denom(tag.attrib['v'], denommapping)

Baptist updated to:
baptist
united_methodist updated to:
methodist
united_methodist;methodist updated to:
methodist
united_methodist updated to:
methodist


* Create dictionary of sport values

In [113]:
def is_sport(elem):
    return (elem.attrib['k'] == "sport")

def audit_sports(osmfile):

    osm_file = open(osmfile, "r")
    sports = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_sport(tag):
                    if tag.attrib['v'] not in sports:
                        sports[tag.attrib['v']] = 1
                    else:
                        sports[tag.attrib['v']] += 1
    return sports

audit_sports(OSM_FILE)

{'american_football': 63,
 'athletics': 2,
 'baseball': 307,
 'basketball': 94,
 'beachvolleyball': 5,
 'crossfit': 2,
 'disc_golf': 1,
 'equestrian': 1,
 'fitness': 2,
 'football': 6,
 'golf': 43,
 'hockey': 1,
 'horseshoes': 1,
 'ice_hockey': 1,
 'karting': 2,
 'lacrosse': 1,
 'martial_arts': 1,
 'motor': 5,
 'multi': 4,
 'racquet': 2,
 'rock_climbing_wall': 1,
 'running': 11,
 'skateboard': 4,
 'soccer': 98,
 'soccer,_crossfit': 1,
 'softball': 16,
 'swimming': 39,
 'tennis': 200,
 'volleyball': 12}

* Replace faulty sport values

In [115]:
sportsmapping = { "multi": "athletics",
                  "beachvolleyball": "volleyball",
                  "football": "american_football",
                  "ice_hockey": "hockey",
                  "karting": "motor",
                  "soccer,_crossfit": "soccer",
            }               

def update_name_sport(name, sportsmapping):
    if name in sportsmapping:
        print name, "updated to:"
        name = sportsmapping[name]
        print name
        return name

    else:
        return name

for event, element in ET.iterparse(OSM_FILE, events=("start",)):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "sport":
                tag.attrib['v'] = update_name_sport(tag.attrib['v'], sportsmapping)

multi updated to:
athletics
soccer,_crossfit updated to:
soccer
ice_hockey updated to:
hockey
football updated to:
american_football
football updated to:
american_football
football updated to:
american_football
multi updated to:
athletics
football updated to:
american_football
football updated to:
american_football
beachvolleyball updated to:
volleyball
football updated to:
american_football
karting updated to:
motor
karting updated to:
motor
beachvolleyball updated to:
volleyball
beachvolleyball updated to:
volleyball
beachvolleyball updated to:
volleyball
beachvolleyball updated to:
volleyball
multi updated to:
athletics
multi updated to:
athletics


* Create dictionary of cuisine values

In [116]:
def is_cuisine(elem):
    return (elem.attrib['k'] == "cuisine")

def audit_cuisine(osmfile):

    osm_file = open(osmfile, "r")
    cuisines = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_cuisine(tag):
                        if tag.attrib['v'] not in cuisines:
                            cuisines[tag.attrib['v']] = 1
                        else:
                            cuisines[tag.attrib['v']] += 1
                        
    return cuisines

audit_cuisine(OSM_FILE)

{'American;Mexican': 1,
 'Asian': 1,
 'Asian_Fusion': 1,
 'Bar': 1,
 'Burrito,_Taco, Salad, Chips': 1,
 'Chinese': 1,
 'Deli': 1,
 'Ice_Cream': 1,
 'Ice_cream': 2,
 'Italian': 1,
 'Japanese_Sushi_Hibachi': 1,
 'Korean': 1,
 'Lebanese-American food': 1,
 'Lebanese_/_Mideastern': 1,
 'Pasta': 1,
 'Polynesian': 1,
 'Steak': 1,
 'Steakhouse_chain': 1,
 'Vegan': 1,
 'Vietnamese': 1,
 'Vietnamese,_Cambodian': 1,
 'american': 30,
 'american_french': 1,
 'asian': 16,
 'australian': 1,
 'bagel': 1,
 'barbecue': 4,
 'bavarian;ice_cream': 1,
 'beer;wings': 1,
 'breakfast': 3,
 'buffet': 1,
 'burger': 75,
 'burger,_shakes': 1,
 'caribbean': 1,
 'chicken': 14,
 'chinese': 22,
 'chinese;vietnamese': 1,
 'coffee_shop': 30,
 'dessert': 1,
 'diner': 2,
 'donut': 2,
 'donuts': 2,
 'french': 4,
 'fusion': 1,
 'fusion;coffee': 1,
 'german': 3,
 'greek': 9,
 'hawaiian': 1,
 'home_cooking': 2,
 'ice_cream': 9,
 'ice_cream,_frozen_yogurt': 1,
 'indian': 7,
 'international': 3,
 'italian': 26,
 'italian._pizz

* Replace faulty cuisine values

In [118]:
cuisinemapping = { "American;Mexican": "mexican",
                   "Vietnamese,_Cambodian": "vietnamese",
                   "bavarian;ice_cream": "ice_cream",
                   "Asian_Fusion": "asian",
                   "Burrito,_Taco, Salad, Chips": "mexican",
                   "Ice_Cream": "ice_cream",
                   "Deli": "sandwich",
                   "Japanese_Sushi_Hibachi": "japanese",
                   "sushi": "japanese",
                   "japanese_steakhouse": "japanese",
                   "Lebanese-American food": "lebanese",
                   "Lebanese_/_Mideastern": "lebanese",
                   "Pasta": "italian",
                   "pasta": "italian",
                   "Steakhouse_chain": "steak",
                   "american_french": "french",
                   "beer;wings": "wings",
                   "bagel": "breakfast",
                   "burger,_shakes": "burger",
                   "chinese;vietnamese": "chinese",
                   "donut": "breakfast",
                   "donuts": "breakfast",
                   "fusion;coffee": "coffee_shop",
                   "home_cooking": "american",
                   "ice_cream,_frozen_yogurt": "ice_cream",
                   "italian._pizza,_bbq,_regional": "italian",
                   "subs": "sandwich"
            }

               

def update_name_cuisine(name, cuisinemapping):
    if name in cuisinemapping:
        print name, 'updated to:'
        name = cuisinemapping[name]
        print name
        return name

    else:
        return first_lower(name)

for event, element in ET.iterparse(OSM_FILE, events=("start",)):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "cuisine":
                tag.attrib['v'] = update_name_cuisine(tag.attrib['v'], cuisinemapping)

sushi updated to:
japanese
beer;wings updated to:
wings
Deli updated to:
sandwich
japanese_steakhouse updated to:
japanese
bagel updated to:
breakfast
Japanese_Sushi_Hibachi updated to:
japanese
Burrito,_Taco, Salad, Chips updated to:
mexican
Vietnamese,_Cambodian updated to:
vietnamese
subs updated to:
sandwich
donuts updated to:
breakfast
Asian_Fusion updated to:
asian
ice_cream,_frozen_yogurt updated to:
ice_cream
burger,_shakes updated to:
burger
subs updated to:
sandwich
Pasta updated to:
italian
home_cooking updated to:
american
sushi updated to:
japanese
home_cooking updated to:
american
pasta updated to:
italian
Lebanese-American food updated to:
lebanese
sushi updated to:
japanese
bavarian;ice_cream updated to:
ice_cream
Lebanese_/_Mideastern updated to:
lebanese
fusion;coffee updated to:
coffee_shop
chinese;vietnamese updated to:
chinese
American;Mexican updated to:
mexican
Ice_Cream updated to:
ice_cream
sushi updated to:
japanese
italian._pizza,_bbq,_regional updated to:
it

* Create dictionary of maxspeed values

In [119]:
def is_speed(elem):
    return (elem.attrib['k'] == "maxspeed")

def audit_speeds(osmfile):

    osm_file = open(osmfile, "r")
    speeds = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_speed(tag):
                        if tag.attrib['v'] not in speeds:
                            speeds[tag.attrib['v']] = 1
                        else:
                            speeds[tag.attrib['v']] += 1
                        
    return speeds



audit_speeds(OSM_FILE)

{'10 mph': 20,
 '105': 71,
 '129870342181342860 mph': 1,
 '15 mph': 18,
 '20 mph': 14,
 '25': 6,
 '25 mph': 238,
 '30 mph': 1,
 '35 mph': 173,
 '40': 1,
 '40 mph': 44,
 '45': 1,
 '45 mph': 162,
 '50 mph': 30,
 '55 mph': 313,
 '56': 4,
 '60 mph': 16,
 '64': 2,
 '65 mph': 621,
 '70': 1,
 '70 mph': 30,
 '72': 21,
 '89': 15}

* Replace faulty maxspeed values

In [120]:
OSM_FILE = "Columbus.osm"

def update_name_maxspeed(name): 
    if len(name) > 7:   #remove blatantly bad speed limit 
        name = "Bad Speed"
        return name
    if 'mph' in name:
        return name
    else:
        print name, 'updated to:'  #make uniform by ensuring mph at end
        name = name.strip() + ' mph'
        print name
        return name

for event, element in ET.iterparse(OSM_FILE, events=("start",)):
    if element.tag == "node" or element.tag == "way":
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "maxspeed" and len(tag.attrib['v']) > 7:
                continue
            else:
                if tag.attrib['k'] == "maxspeed":
                    tag.attrib['v'] = update_name_maxspeed(tag.attrib['v'])

56 updated to:
56 mph
89 updated to:
89 mph
40 updated to:
40 mph
89 updated to:
89 mph
89 updated to:
89 mph
105 updated to:
105 mph
105 updated to:
105 mph
25 updated to:
25 mph
45 updated to:
45 mph
70 updated to:
70 mph
25 updated to:
25 mph
89 updated to:
89 mph
56 updated to:
56 mph
25 updated to:
25 mph
89 updated to:
89 mph
105 updated to:
105 mph
64 updated to:
64 mph
64 updated to:
64 mph
105 updated to:
105 mph
72 updated to:
72 mph
25 updated to:
25 mph
25 updated to:
25 mph
72 updated to:
72 mph
105 updated to:
105 mph
105 updated to:
105 mph
105 updated to:
105 mph
105 updated to:
105 mph
56 updated to:
56 mph
72 updated to:
72 mph
72 updated to:
72 mph
72 updated to:
72 mph
89 updated to:
89 mph
89 updated to:
89 mph
89 updated to:
89 mph
89 updated to:
89 mph
89 updated to:
89 mph
89 updated to:
89 mph
105 updated to:
105 mph
105 updated to:
105 mph
105 updated to:
105 mph
105 updated to:
105 mph
105 updated to:
105 mph
72 updated to:
72 mph
72 updated to:
72 mph
72 upd

# Main Function

In [1]:
import cerberus
import xml.etree.cElementTree as ET
import schema
import csv
import codecs
import pprint
import re
from collections import defaultdict
OSM_PATH = "Columbus.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

#variables to call in shape_element
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

#mapping to call in shape_element
streetmapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road",
            "Rd": "Road",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "S": "South",
            "SE": "Southeast",
            "SW": "Southwest",
            "N": "North",
            "NW": "Northwest",
            "NE": "Northeast",
            "Pkwy": "Parkway",
            "Rd 101": "Road",
            "St Suite 105": "Street",
            "Rd 250": "Road",
            "Rd 2D": "Road",
            "Rd #320": "Road",
            "8877": "Bad Address",
            "8897": " Bad Address"
            }

citymapping = { "Columbus, OH": "Columbus",
                "Dublin, OH": "Dublin",
                "Reynoldsburh": "Reynoldsburg",
                "Pickertington": "Pickerington"
            }

denommapping = { "united_methodist;methodist": "methodist",
                 "united_methodist": "methodist"
            }

sportsmapping = { "multi": "athletics",
                  "beachvolleyball": "volleyball",
                  "football": "american_football",
                  "ice_hockey": "hockey",
                  "karting": "motor",
                  "soccer,_crossfit": "soccer",
            }

cuisinemapping = { "American;Mexican": "mexican",
                   "Vietnamese,_Cambodian": "vietnamese",
                   "bavarian;ice_cream": "ice_cream",
                   "Asian_Fusion": "asian",
                   "Burrito,_Taco, Salad, Chips": "mexican",
                   "Ice_Cream": "ice_cream",
                   "Deli": "sandwich",
                   "Japanese_Sushi_Hibachi": "japanese",
                   "sushi": "japanese",
                   "japanese_steakhouse": "japanese",
                   "Lebanese-American food": "lebanese",
                   "Lebanese_/_Mideastern": "lebanese",
                   "Pasta": "italian",
                   "pasta": "italian",
                   "Steakhouse_chain": "steak",
                   "american_french": "french",
                   "beer;wings": "wings",
                   "bagel": "breakfast",
                   "burger,_shakes": "burger",
                   "chinese;vietnamese": "chinese",
                   "donut": "breakfast",
                   "donuts": "breakfast",
                   "fusion;coffee": "coffee_shop",
                   "home_cooking": "american",
                   "ice_cream,_frozen_yogurt": "ice_cream",
                   "italian._pizza,_bbq,_regional": "italian",
                   "subs": "sandwich"
            }

#Functions to call in shape_element cleaning process
def update_name_street(name, streetmapping):
    if name == "E Dublin Granville Rd #101":
        name = "E Dublin Granville Road"
    elif name == "Mill St Suite 105":
        name = "Mill Street"
    elif name == "Chambers Rd #250":
        name = "Chambers Road"
    elif name == "Copeland Mill Rd #2D":
        name = "Copeland Mill Road"
    elif name == "W Old Wilson Bridge Rd #320":
        name = "W Old Wilson Bridge Road"
    
    else:
        m = street_type_re.search(name)
        street_type = m.group()
        if street_type in streetmapping.keys():
            name = re.sub(m.group(), streetmapping[m.group()],name)
    return name

def update_zips(name):
    if len(name) < 5:
        #print name, 'updated to:'
        name = "Bad Zip"
        #print name
        return name
    elif len(name) > 5:
        #print name, 'updated to:'
        if re.search('[0-9]{5}', name):
            updated_name = re.findall('[0-9]{5}', name)
            name = updated_name[0]
            #print name
            return name
    
    else:
        return name

def update_name_city(name, citymapping):
    if name in citymapping:
        #print name, "updated to:"
        name = citymapping[name]
        if name.islower():
            name = name.capitalize()
        #print name
        return name

    else:
        if name.islower():
            name = name.capitalize()
        return name

def update_name_denom(name, denommapping):
    if name.istitle():
        #print name, 'updated to:'
        name = name.lower()
        #print name
        return name
    elif name in denommapping:
        #print name, "updated to:"
        name = denommapping[name]
        #print name
        return name

    else:
        return name

def first_lower(s):
    return s[0].lower() + s[1:]

def update_name_sport(name, sportsmapping):
    if name in sportsmapping:
        #print name, "updated to:"
        name = sportsmapping[name]
        #print name
        return name

    else:
        return name

def update_name_cuisine(name, cuisinemapping):
    if name in cuisinemapping:
        #print name, 'updated to:'
        name = cuisinemapping[name]
        #print name
        return name

    else:
        return first_lower(name)

def update_name_maxspeed(name):
    if len(name) > 7:
        name = "Bad Speed"
        return name
    if 'mph' in name:
        return name
    else:
        #print name, 'updated to:'
        name = name.strip() + ' mph'
        #print name
        return name

def right_key(k):
    index = k.find(':')
    types = k[:index]
    k = k[index + 1:]
    return k,types

#Main Function
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_atts = {}
    way_atts = {}
    way_nodes = []
    tags = []
    
    if element.tag == 'node': #fill dictionary with k/v pairs from NODE_FIELDS
        for i in node_attr_fields:
            node_atts[i] = element.attrib[i]

    if element.tag == 'way':
        for i in way_attr_fields:
            way_atts[i] = element.attrib[i]

    for tag in element.iter("tag"): #loop through tags looking for problem values
        dic = {}
        attributes = tag.attrib
        if tag.attrib['k'] == "addr:street":
            tag.attrib['v'] = update_name_street(tag.attrib['v'], streetmapping)
        elif tag.attrib['k'] == "addr:city":
            tag.attrib['v'] = update_name_city(tag.attrib['v'], citymapping)
        elif tag.attrib['k'] == "addr:postcode":
            tag.attrib['v'] = update_zips(tag.attrib['v'])
        elif tag.attrib['k'] == "denomination":
            tag.attrib['v'] = update_name_denom(tag.attrib['v'], denommapping)
        elif tag.attrib['k'] == "sport":
            tag.attrib['v'] = update_name_sport(tag.attrib['v'], sportsmapping)
        elif tag.attrib['k'] == "cuisine":
            tag.attrib['v'] = update_name_cuisine(tag.attrib['v'], cuisinemapping)
        elif tag.attrib['k'] == "maxspeed":
            tag.attrib['v'] = update_name_maxspeed(tag.attrib['v'])      
        
        
        if problem_chars.search(tag.attrib['k']):
            continue

        if element.tag == 'node': #add node id for attributes
            dic['id'] = node_atts['id']
        else:
            dic['id'] = way_atts['id'] #add way id for attributes

        dic['value'] = attributes['v'] #value of key for each type

        colon_k=LOWER_COLON.search(tag.attrib['k'])
        
        if colon_k:
            #print colon_k.group(0)
            #print tag.attrib['k']
            dic['key'],dic['type'] = right_key(tag.attrib['k']) #call function to split at colon
        else:
            dic['key'] = attributes['k'] #assign regular that there was no colon problem
            dic['type'] = 'regular'

        tags.append(dic)

    if element.tag == 'way':
        position = 0
        for nd in element.iter("nd"): #loop through nd child tags numbering them
            way_node_dic = {}
            way_node_dic['id'] = way_atts['id']
            way_node_dic['node_id'] = nd.attrib['ref']
            way_node_dic['position'] = position
            position = position + 1
            way_nodes.append(way_node_dic)

    if element.tag == 'node':       #process the above for node tags for final formatting
        return {'node': node_atts, 'node_tags': tags}

    elif element.tag == 'way':      #process the above for way tags for final formatting
        return {'way': way_atts, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)

        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

* Next 5 create tables in database

In [5]:
import sqlite3
import csv
from pprint import pprint

sqlite_file = 'columbusaudit.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS nodes')
conn.commit()

cur.execute('''
    CREATE TABLE nodes(id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT)
''')
conn.commit()

with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['lat'],i['lon'], i['user'].decode("utf-8"), i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
# insert data
cur.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

#cur.execute('SELECT * FROM nodes')
#all_rows = cur.fetchall()
#print('1):')
#pprint(all_rows)

conn.close()

In [ ]:
sqlite_file = 'columbusaudit.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS nodes_tags')
conn.commit()

cur.execute('''
    CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id))
''')
conn.commit()

with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'],i['value'].decode("utf-8"), i['type']) for i in dr]
    
# insert data
cur.executemany("INSERT INTO nodes_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

#cur.execute('SELECT * FROM nodes_tags')
#all_rows = cur.fetchall()
#print('1):')
#pprint(all_rows)

conn.close()

In [12]:
sqlite_file = 'columbusaudit.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS ways')
conn.commit()

cur.execute('''
    CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT)
''')
conn.commit()

with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['user'].decode("utf-8"), i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]
    
# insert data
cur.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

#cur.execute('SELECT * FROM ways')
#all_rows = cur.fetchall()
#print('1):')
#pprint(all_rows)

conn.close()

In [14]:
sqlite_file = 'columbusaudit.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS ways_tags')
conn.commit()

cur.execute('''
    CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id))
''')
conn.commit()

with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['key'],i['value'].decode("utf-8"), i['type']) for i in dr]
    
# insert data
cur.executemany("INSERT INTO ways_tags(id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

#cur.execute('SELECT * FROM ways_tags')
#all_rows = cur.fetchall()
#print('1):')
#pprint(all_rows)

conn.close()

In [15]:
sqlite_file = 'columbusaudit.db'

conn = sqlite3.connect(sqlite_file)

cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS ways_nodes')
conn.commit()

cur.execute('''
    CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id))
''')
conn.commit()

with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'], i['node_id'],i['position']) for i in dr]
    
# insert data
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)
# commit the changes
conn.commit()

#cur.execute('SELECT * FROM ways_nodes')
#all_rows = cur.fetchall()
#print('1):')
#pprint(all_rows)

conn.close()

In [16]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT COUNT(*) FROM nodes;
''')
results = cursor.fetchall()
print results
conn.close

[(675741,)]


<function close>

In [17]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT COUNT(*) FROM ways;
''')
results = cursor.fetchall()
print results
conn.close

[(75957,)]


<function close>

In [19]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT Count(DISTINCT both.user)
    FROM (SELECT user FROM nodes
    UNION ALL SELECT user FROM ways) as both;
''')
results = cursor.fetchall()
print results
conn.close

[(765,)]


<function close>

In [91]:
import pandas as pd 

conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT both.value, COUNT(*) as Total 
    FROM (SELECT * FROM nodes_tags UNION ALL 
    SELECT * FROM ways_tags) as both
    WHERE both.key == 'city'
    GROUP BY both.value
    ORDER BY Total DESC
    LIMIT 10;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close


                 0    1
0         Columbus  803
1           Dublin  329
2  Upper Arlington   98
3         Hilliard   82
4     Pickerington   58
5         Galloway   29
6     Reynoldsburg   27
7        Pataskala   22
8       New Albany   21
9      Worthington   21


<function close>

In [125]:
import pandas as pd 

conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT both.value, COUNT(*) as Total 
    FROM (SELECT * FROM nodes_tags UNION ALL 
    SELECT * FROM ways_tags) as both
    WHERE both.key == 'postcode'
    GROUP BY both.value
    ORDER BY Total DESC
    LIMIT 5;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close


       0    1
0  43017  319
1  43221  111
2  43210   89
3  43026   86
4  43215   79


<function close>

* ElaineNewport put a Marathon gas station in Adamsville, Alabama.  Deemed not significant enough to put in final write-up submission, can easily remove node before reinserting cleaned data to OpenStreetMap.

In [88]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT * 
    FROM nodes_tags JOIN nodes
    on nodes_tags.id = nodes.id
    WHERE nodes_tags.key == 'postcode' and nodes_tags.value = 35005;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close

           0         1      2     3           4         5          6   \
0  1278853280  postcode  35005  addr  1278853280  39.98465 -82.815342   

              7        8   9         10                    11  
0  ElaineNewport  2564091   3  28033823  2015-01-10T05:07:17Z  


<function close>

In [90]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT * 
    FROM nodes_tags 
    WHERE id == 1278853280;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close

            0              1                           2        3
0  1278853280           city                  Adamsville     addr
1  1278853280    housenumber                       35005     addr
2  1278853280       postcode                       35005     addr
3  1278853280          state                     Alabama     addr
4  1278853280         street           East Broad Street     addr
5  1278853280        amenity                        fuel  regular
6  1278853280           name                    Marathon  regular
7  1278853280  opening_hours                      sasqsq  regular
8  1278853280       operator                 sqsqsqsqsqs  regular
9  1278853280      wikipedia  en:1995 World Marathon Cup  regular


<function close>

In [124]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT value, COUNT(*) as Total 
    FROM nodes_tags
    WHERE key == 'denomination'
    GROUP BY value
    ORDER BY Total DESC
    LIMIT 5;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close

              0    1
0       baptist  127
1     methodist   52
2      lutheran   38
3      catholic   21
4  presbyterian   17


<function close>

In [123]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT both.value, COUNT(*) as Total 
    FROM (SELECT * FROM nodes_tags UNION ALL 
    SELECT * FROM ways_tags) as both
    WHERE both.key == 'sport'
    GROUP BY both.value
    ORDER BY Total DESC
    LIMIT 5;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close

                   0    1
0           baseball  311
1             tennis  203
2             soccer  100
3         basketball   94
4  american_football   70


<function close>

In [122]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT both.value, COUNT(*) as Total 
    FROM (SELECT * FROM nodes_tags UNION ALL 
    SELECT * FROM ways_tags)
    as both
    WHERE both.key == 'cuisine'
    GROUP BY both.value
    ORDER BY Total DESC
    LIMIT 5;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close

          0   1
0    burger  77
1   mexican  56
2     pizza  48
3  sandwich  38
4  american  32


<function close>

In [51]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()

cursor.execute('''
    CREATE view both as
    SELECT * FROM nodes_tags UNION ALL 
    SELECT * FROM ways_tags;
''')

In [52]:
cursor.execute('''
    CREATE view coffee_shop as
    SELECT id
    FROM both
    WHERE both.key == 'cuisine' and both.value == 'coffee_shop';
''')

In [96]:
cursor.execute('''
    SELECT both.value, COUNT(*) from both
    JOIN coffee_shop
    on both.id = coffee_shop.id
    WHERE both.key = 'name' and both.value = 'Starbucks'
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close

           0   1
0  Starbucks  11


<function close>

In [81]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT both.value, Count(*)   
    FROM (SELECT * FROM nodes_tags UNION ALL 
    SELECT * FROM ways_tags) as both
    WHERE both.value == 'Starbucks';
''')

results = cursor.fetchall()
print results
conn.close

[(u'Starbucks', 23)]


<function close>

In [97]:
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT both.value, COUNT(*) as Total 
    FROM (SELECT * FROM nodes_tags UNION ALL 
    SELECT * FROM ways_tags)
    as both
    WHERE both.key == 'maxspeed'
    GROUP BY both.value
    ORDER BY Total DESC
    Limit 6;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close

         0    1
0   65 mph  641
1   55 mph  321
2   25 mph  249
3   35 mph  180
4   45 mph  168
5  105 mph   74


<function close>

In [71]:
#use both view as a table to join
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()
cursor.execute('''
    SELECT ways.user, COUNT(*) as Total 
    FROM both JOIN ways
    on both.id = ways.id
    WHERE both.key = 'maxspeed' and both.value = '105 mph'
    GROUP BY ways.user
    ORDER BY Total DESC;
''')

results = cursor.fetchall()
df = pd.DataFrame(results)
print df
conn.close

              0   1
0   Vid the Kid  37
1           NE2  32
2          cl94   3
3  Sunfishtommy   2


<function close>